<a href="https://colab.research.google.com/github/Aditya1234M/Nutrit/blob/main/NutritionAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1 — GPU setup (Colab T4)

import tensorflow as tf
import subprocess
import os

# Print TensorFlow version
print("TensorFlow version:", tf.__version__)

# Check GPU using TensorFlow
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    print("\n🔥 GPU Detected!")
    for gpu in gpus:
        print("GPU:", gpu)

    # Enable safe memory growth
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print("✔ Memory growth enabled.")
    except Exception as e:
        print("⚠ Could not set memory growth:", e)
else:
    print("\n❌ No GPU detected. Training will run on CPU (slower).")

# Check system GPU info (NVIDIA-SMI)
print("\n=== NVIDIA-SMI Output ===")
gpu_info = subprocess.getoutput("nvidia-smi")
print(gpu_info)


TensorFlow version: 2.19.0

🔥 GPU Detected!
GPU: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
✔ Memory growth enabled.

=== NVIDIA-SMI Output ===
Sun Nov 23 15:41:42 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              9W /   70W |       2MiB /  15360MiB

In [ ]:
# Cell 2 — Load Food-101 from TFDS
import tensorflow_datasets as tfds

# loads train and validation splits, as_supervised gives (image, label) pairs
(food101_train_raw, food101_val_raw), food101_info = tfds.load(
    "food101",
    split=["train", "validation"],
    as_supervised=True,
    with_info=True
)

num_food101_classes = food101_info.features["label"].num_classes
print("Food-101 classes:", num_food101_classes)


Food-101 classes: 101


In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("iamsouravbanerjee/indian-food-images-dataset")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'indian-food-images-dataset' dataset.
Path to dataset files: /kaggle/input/indian-food-images-dataset


In [ ]:
import os

print("Root:", "/kaggle/input/indian-food-images-dataset")
print(os.listdir("/kaggle/input/indian-food-images-dataset"))

# If inside there is a subfolder, list it too:
sub1 = "/kaggle/input/indian-food-images-dataset/" + os.listdir("/kaggle/input/indian-food-images-dataset")[0]
print("Subfolder:", sub1)
print(os.listdir(sub1))


Root: /kaggle/input/indian-food-images-dataset
['Indian Food Images', 'List of Indian Foods.txt']
Subfolder: /kaggle/input/indian-food-images-dataset/Indian Food Images
['Indian Food Images']


In [ ]:
# Cell 3 — Indian dataset from KaggleHub path

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# IMPORTANT — Correct path from KaggleHub
INDIAN_PATH = "/kaggle/input/indian-food-images-dataset/Indian Food Images/Indian Food Images"


IMG_SIZE = 300
BATCH = 16

# Image generator
indian_gen = ImageDataGenerator(
    rescale=1/255.0,
    validation_split=0.2
)

# Training generator
indian_train_gen = indian_gen.flow_from_directory(
    INDIAN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode="sparse",   # returns integer labels
    subset="training"
)

# Validation generator
indian_val_gen = indian_gen.flow_from_directory(
    INDIAN_PATH,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH,
    class_mode="sparse",
    subset="validation"
)

print("Indian classes:", indian_train_gen.num_classes)


Found 3200 images belonging to 80 classes.
Found 800 images belonging to 80 classes.
Indian classes: 80


In [ ]:
# Cell 4 — preprocess food101 images (resize + normalize) and batch
import tensorflow as tf

def preprocess_food101(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

food101_train = food101_train_raw.map(preprocess_food101).shuffle(2048).batch(BATCH).prefetch(tf.data.AUTOTUNE)
food101_val   = food101_val_raw.map(preprocess_food101).batch(BATCH).prefetch(tf.data.AUTOTUNE)

# quick sanity prints
print("Food101 train batches:", tf.data.experimental.cardinality(food101_train).numpy())
print("Food101 val batches:", tf.data.experimental.cardinality(food101_val).numpy())
print("Indian train steps (batches):", len(indian_train_gen))
print("Indian val steps (batches):", len(indian_val_gen))


Food101 train batches: 4735
Food101 val batches: 1579
Indian train steps (batches): 200
Indian val steps (batches): 50


In [ ]:
# Count classes from both datasets

# Correct number of classes for Food-101
num_food101_classes = food101_info.features["label"].num_classes

# Correct number of classes for Indian dataset
num_indian_classes = indian_train_gen.num_classes   # this exists because you used ImageDataGenerator

print("Food101 classes:", num_food101_classes)
print("Indian classes:", num_indian_classes)


Food101 classes: 101
Indian classes: 80


In [ ]:
# Cell 5 — Build model with one shared backbone and two heads
from tensorflow.keras import layers, Model

base_model = tf.keras.applications.EfficientNetB4(
    include_top=False,
    weights="imagenet",
    pooling="avg",
    input_shape=(IMG_SIZE, IMG_SIZE, 3)
)
base_model.trainable = False  # start with frozen backbone

inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name="input_image")
features = base_model(inputs, training=False)

food101_head = layers.Dense(num_food101_classes, activation="softmax", name="food101_head")(features)
indian_head  = layers.Dense(num_indian_classes, activation="softmax", name="indian_head")(features)

model = Model(inputs=inputs, outputs=[food101_head, indian_head], name="multihead_food_model")
model.summary()


Model: "multihead_food_model"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_image         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetb4      │ (None, 1792)      │ 17,673,823 │ input_image[0][0] │
│ (Functional)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ food101_head        │ (None, 101)       │    181,093 │ efficientnetb4[0… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ indian_head (Dense) │ (None, 80)        │    143,440 │ efficientnetb4[0… │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 17,998,356 (68.66 MB)

 Trainable params: 324,533 (1.24 MB)

 Non-trainable params: 17,673,823 (67.42 MB)

In [ ]:
# Cell 7 — Compile the model

losses = {
    "food101_head": "sparse_categorical_crossentropy",
    "indian_head": "sparse_categorical_crossentropy",
}

metrics = {
    "food101_head": ["accuracy"],
    "indian_head": ["accuracy"],
}

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-4),
    loss=losses,
    metrics=metrics
)

print("Model compiled successfully.")


Model compiled successfully.


In [ ]:
# Convert Indian DirectoryIterator → tf.data.Dataset

def indian_gen_to_dataset(gen):
    output_types = (tf.float32, tf.float32)       # (images, labels)
    output_shapes = (tf.TensorShape([None, IMG_SIZE, IMG_SIZE, 3]),
                     tf.TensorShape([None]))      # labels shape

    return tf.data.Dataset.from_generator(
        lambda: gen,
        output_types=output_types,
        output_shapes=output_shapes
    )


In [ ]:
indian_train_ds = indian_gen_to_dataset(indian_train_gen)
indian_val_ds   = indian_gen_to_dataset(indian_val_gen)


In [ ]:
def preprocess_food101(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [ ]:
# Wrap both datasets in the format the multi-head model expects

def wrap_indian(images, labels):
    return images, {
        "food101_head": tf.zeros_like(labels),
        "indian_head": labels
    }

def wrap_food101(image, label):
    return image, {
        "food101_head": label,
        "indian_head": tf.zeros_like(label)
    }


In [ ]:
# Food-101 datasets (already tf.data)
food101_train = (
    food101_train_raw
    .map(preprocess_food101, num_parallel_calls=tf.data.AUTOTUNE)
    .map(wrap_food101, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH)
    .prefetch(tf.data.AUTOTUNE)
)

food101_val = (
    food101_val_raw
    .map(preprocess_food101, num_parallel_calls=tf.data.AUTOTUNE)
    .map(wrap_food101, num_parallel_calls=tf.data.AUTOTUNE)
    .batch(BATCH)
    .prefetch(tf.data.AUTOTUNE)
)


# Indian datasets converted from generator
indian_train = indian_train_ds.map(wrap_indian).prefetch(tf.data.AUTOTUNE)
indian_val   = indian_val_ds.map(wrap_indian).prefetch(tf.data.AUTOTUNE)


In [ ]:
# Cell 9 — Loss weights

loss_weights_food101 = {
    "food101_head": 1.0,
    "indian_head": 0.0
}

loss_weights_indian = {
    "food101_head": 0.0,
    "indian_head": 1.0
}


In [ ]:
for images, labels in food101_train.take(1):
    print("Food101 shape:", images.shape)

for images, labels in indian_train.take(1):
    print("Indian shape:", images.shape)


Food101 shape: (16, 300, 300, 3)
Indian shape: (16, 300, 300, 3)


In [ ]:
for batch in food101_val.take(2):
    img, lbl = batch
    print("Food101 val batch OK:", img.shape)
    break

for batch in indian_val.take(2):
    img, lbl = batch
    print("Indian val batch OK:", img.shape)
    break


Food101 val batch OK: (16, 300, 300, 3)
Indian val batch OK: (16, 300, 300, 3)


In [ ]:
# ============================
# 🔒 STAGE 1: FREEZE BACKBONE
# ============================

# Freeze ALL layers of EfficientNetB4
for layer in base_model.layers:
    layer.trainable = False

print("Backbone frozen. Training only final classification heads.")

# Compile with a higher LR for head training
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss={
        "food101_head": "sparse_categorical_crossentropy",
        "indian_head": "sparse_categorical_crossentropy"
    },
    metrics=["accuracy"]
)


Backbone frozen. Training only final classification heads.


In [ ]:
# Freeze backbone
base_model.trainable = False

# Compile for stage 1
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss={
        "food101_head": tf.keras.losses.SparseCategoricalCrossentropy(),
        "indian_head": tf.keras.losses.SparseCategoricalCrossentropy(),
    },
    metrics={
        "food101_head": "accuracy",
        "indian_head": "accuracy"
    }
)

EPOCHS_STAGE1 = 3

history1 = model.fit(
    food101_train.zip(indian_train),
    validation_data=food101_val.zip(indian_val),
    epochs=EPOCHS_STAGE1
)


Epoch 1/3
    244/Unknown 86s 122ms/step - food101_head_accuracy: 0.9496 - food101_head_loss: 0.6500 - indian_head_accuracy: 0.0081 - indian_head_loss: 4.5105 - loss: 5.1605

KeyboardInterrupt: 

In [ ]:
# Cell 10 — Correct multi-task training loop

EPOCHS = 5

# steps per epoch for each dataset
steps_food101 = food101_train_raw.cardinality().numpy() // BATCH
steps_indian  = indian_train_gen.samples // BATCH

# choose the larger one
steps_per_epoch = 200

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    food101_iter = iter(food101_train)
    indian_iter = iter(indian_train)

    for step in range(steps_per_epoch):

        # ---- FOOD101 batch ----
        try:
            x_f, y_f = next(food101_iter)
            model.train_on_batch(x_f, y_f)
        except StopIteration:
            pass

        # ---- INDIAN batch ----
        try:
            x_i, y_i = next(indian_iter)
            model.train_on_batch(x_i, y_i)
        except StopIteration:
            pass
        if step % 20 == 0:
            print(f"Step {step}/{steps_per_epoch}")

    # ---- Validation ----
    print("Validating...")

    val_food = model.evaluate(food101_val.take(50), verbose=0)
    val_ind  = model.evaluate(indian_val.take(5), verbose=0)

    print(f"Food101 Val Loss/Acc: {val_food}")
    print(f"Indian Val Loss/Acc : {val_ind}")



Epoch 1/5
Step 0/200
Step 20/200
Step 40/200
Step 60/200
Step 80/200
Step 100/200
Step 120/200
Step 140/200
Step 160/200
Step 180/200
Validating...


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


Food101 Val Loss/Acc: [6.061058521270752, 5.437210559844971, 0.6238466501235962, 0.008750000037252903, 1.0]
Indian Val Loss/Acc : [5.769024848937988, 0.6395354866981506, 5.129488945007324, 1.0, 0.0]

Epoch 2/5
Step 0/200
Step 20/200
Step 40/200
Step 60/200
Step 80/200
Step 100/200
Step 120/200
Step 140/200
Step 160/200
Step 180/200
Validating...
Food101 Val Loss/Acc: [6.0302042961120605, 5.401789665222168, 0.6284130215644836, 0.008750000037252903, 1.0]
Indian Val Loss/Acc : [5.586277961730957, 0.6384649276733398, 4.947813510894775, 1.0, 0.03750000149011612]

Epoch 3/5
Step 0/200
Step 20/200
Step 40/200
Step 60/200
Step 80/200
Step 100/200
Step 120/200
Step 140/200
Step 160/200
Step 180/200
Validating...
Food101 Val Loss/Acc: [6.015382289886475, 5.4053778648376465, 0.6100038290023804, 0.008750000037252903, 1.0]
Indian Val Loss/Acc : [5.869945049285889, 0.6110502481460571, 5.258894920349121, 1.0, 0.0]

Epoch 4/5
Step 0/200
Step 20/200
Step 40/200
Step 60/200
Step 80/200
Step 100/200
Step

In [ ]:
model.save("multihead_food_model.h5")
print("Saved.")


Saved.


In [ ]:
# Unfreeze last 50 layers
for layer in base_model.layers[-50:]:
    layer.trainable = True

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss=losses,
    metrics=metrics
)

print("Fine-tuning started…")


Fine-tuning started…


In [ ]:
EPOCHS = 5

# steps per epoch for each dataset
steps_food101 = food101_train_raw.cardinality().numpy() // BATCH
steps_indian  = indian_train_gen.samples // BATCH

# choose the larger one
steps_per_epoch = 1000

for epoch in range(EPOCHS):
    print(f"\nEpoch {epoch+1}/{EPOCHS}")

    food101_iter = iter(food101_train)
    indian_iter = iter(indian_train)

    for step in range(steps_per_epoch):

        # ---- FOOD101 batch ----
        try:
            x_f, y_f = next(food101_iter)
            model.train_on_batch(x_f, y_f)
        except StopIteration:
            pass

        # ---- INDIAN batch ----
        try:
            x_i, y_i = next(indian_iter)
            model.train_on_batch(x_i, y_i)
        except StopIteration:
            pass
        if step % 20 == 0:
            print(f"Step {step}/{steps_per_epoch}")

    # ---- Validation ----
    print("Validating...")

    val_food = model.evaluate(food101_val.take(50), verbose=0)
    val_ind  = model.evaluate(indian_val.take(5), verbose=0)

    print(f"Food101 Val Loss/Acc: {val_food}")
    print(f"Indian Val Loss/Acc : {val_ind}")



Epoch 1/5
Step 0/1000
Step 20/1000
Step 40/1000
Step 60/1000
Step 80/1000
Step 100/1000
Step 120/1000
Step 140/1000
Step 160/1000
Step 180/1000
Step 200/1000
Step 220/1000
Step 240/1000
Step 260/1000
Step 280/1000
Step 300/1000
Step 320/1000
Step 340/1000
Step 360/1000
Step 380/1000
Step 400/1000
Step 420/1000
Step 440/1000
Step 460/1000
Step 480/1000
Step 500/1000
Step 520/1000
Step 540/1000
Step 560/1000
Step 580/1000
Step 600/1000
Step 620/1000
Step 640/1000
Step 660/1000
Step 680/1000
Step 700/1000
Step 720/1000
Step 740/1000
Step 760/1000
Step 780/1000
Step 800/1000
Step 820/1000
Step 840/1000
Step 860/1000
Step 880/1000
Step 900/1000
Step 920/1000
Step 940/1000
Step 960/1000
Step 980/1000
Validating...
Food101 Val Loss/Acc: [6.632874011993408, 5.690617561340332, 0.942257285118103, 0.008750000037252903, 0.9987499713897705]
Indian Val Loss/Acc : [5.182485580444336, 0.3831041753292084, 4.799381732940674, 0.987500011920929, 0.0]

Epoch 2/5
Step 0/1000
Step 20/1000
Step 40/1000
Step 

In [ ]:
model.save("multihead_final.h5")


In [ ]:
print("\n🔓 Unfreezing TOP 50 EfficientNet layers...")

# Unfreeze only the top 50 layers
for layer in base_model.layers[:-50]:
    layer.trainable = False

for layer in base_model.layers[-50:]:
    layer.trainable = True

print("Trainable layers:", sum([layer.trainable for layer in base_model.layers]))

# Very small LR for fine-tuning
fine_tune_lr = 3e-5

model.compile(
    optimizer=tf.keras.optimizers.Adam(fine_tune_lr),
    loss={
        "food101_head": tf.keras.losses.SparseCategoricalCrossentropy(),
        "indian_head": tf.keras.losses.SparseCategoricalCrossentropy(),
    },
    metrics={
        "food101_head": "accuracy",
        "indian_head": "accuracy"
    }
)

# Reduce dataset steps (FAST!)
steps_per_epoch = 200     # instead of 4700

EPOCHS_FT = 2            # instead of 6

history3 = model.fit(
    food101_train.zip(indian_train),
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS_FT
)

print("🎉 Fast fine-tuning done!")



🔓 Unfreezing TOP 50 EfficientNet layers...
Trainable layers: 50
Epoch 1/2
200/200 ━━━━━━━━━━━━━━━━━━━━ 80s 115ms/step - food101_head_accuracy: 0.8669 - food101_head_loss: 1.4099 - indian_head_accuracy: 0.0185 - indian_head_loss: 4.4831 - loss: 5.8930
Epoch 2/2
200/200 ━━━━━━━━━━━━━━━━━━━━ 23s 117ms/step - food101_head_accuracy: 1.0000 - food101_head_loss: 0.0241 - indian_head_accuracy: 0.0214 - indian_head_loss: 4.3687 - loss: 4.3928
🎉 Fast fine-tuning done!


In [ ]:
print("\n🔓 Unfreezing ALL EfficientNet layers for final fine-tuning...")

# Unfreeze whole backbone
for layer in base_model.layers:
    layer.trainable = True

print("Trainable layers:", sum([layer.trainable for layer in base_model.layers]))

# --- Very small LR ---
fine_tune_lr = 1e-5

model.compile(
    optimizer=tf.keras.optimizers.Adam(fine_tune_lr),
    loss={
        "food101_head": tf.keras.losses.SparseCategoricalCrossentropy(),
        "indian_head": tf.keras.losses.SparseCategoricalCrossentropy(),
    },
    metrics={
        "food101_head": "accuracy",
        "indian_head": "accuracy"
    }
)

# Proper steps based on dataset size
steps_food101 = food101_train_raw.cardinality().numpy() // BATCH
steps_indian  = indian_train_gen.samples // BATCH

steps_per_epoch = steps_food101    # Food101 dominates

EPOCHS_FT = 2

history3 = model.fit(
    food101_train.zip(indian_train),
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS_FT
)

print("🎉 Fine-tuning complete!")



🔓 Unfreezing ALL EfficientNet layers for final fine-tuning...
Trainable layers: 476
Epoch 1/2
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 1665s 319ms/step - food101_head_accuracy: 0.9916 - food101_head_loss: 0.2058 - indian_head_accuracy: 0.1967 - indian_head_loss: 3.5765 - loss: 3.7823
Epoch 2/2
2640/4734 ━━━━━━━━━━━━━━━━━━━━ 11:06 318ms/step - food101_head_accuracy: 1.0000 - food101_head_loss: 0.0096 - indian_head_accuracy: 0.8593 - indian_head_loss: 0.8892 - loss: 0.8988

In [ ]:
print("\n🔓 Unfreezing ALL EfficientNet layers for final fine-tuning...")

# Unfreeze whole backbone
for layer in base_model.layers:
    layer.trainable = True

print("Trainable layers:", sum([layer.trainable for layer in base_model.layers]))

# --- Very small LR ---
fine_tune_lr = 1e-5

model.compile(
    optimizer=tf.keras.optimizers.Adam(fine_tune_lr),
    loss={
        "food101_head": tf.keras.losses.SparseCategoricalCrossentropy(),
        "indian_head": tf.keras.losses.SparseCategoricalCrossentropy(),
    },
    metrics={
        "food101_head": "accuracy",
        "indian_head": "accuracy"
    }
)

# Proper steps based on dataset size
steps_food101 = food101_train_raw.cardinality().numpy() // BATCH
steps_indian  = indian_train_gen.samples // BATCH

steps_per_epoch = steps_food101    # Food101 dominates

EPOCHS_FT = 6

history3 = model.fit(
    food101_train.zip(indian_train),
    validation_data=food101_val.zip(indian_val),
    steps_per_epoch=steps_per_epoch,
    epochs=EPOCHS_FT
)

print("🎉 Fine-tuning complete!")



🔓 Unfreezing ALL EfficientNet layers for final fine-tuning...
Trainable layers: 476
Epoch 1/6
4734/4734 ━━━━━━━━━━━━━━━━━━━━ 0s 321ms/step - food101_head_accuracy: 0.9799 - food101_head_loss: 0.3038 - indian_head_accuracy: 0.2178 - indian_head_loss: 3.4054 - loss: 3.7092

In [ ]:
model.save("multihead_final.h5")
print("Model saved.")


Model saved.


In [ ]:
model.save("/content/drive/MyDrive/multi_food_model.h5")
